In [1]:
import torch
from MVCNN.models import MVCNN
from PFT import PruningFineTuner
import numpy as np
from copy import deepcopy
import os
from torchvision import models
import numpy as np

In [2]:
def get_size(model):
    param_size = sum(
        param.nelement() * param.element_size() for param in model.parameters()
    )
    buffer_size = sum(
        buffer.nelement() * buffer.element_size() for buffer in model.buffers()
    )
    return (param_size + buffer_size) / 1024**2

In [3]:
def get_model():
    model = MVCNN.SVCNN('SVCNN')
    model.load_state_dict(torch.load('mvcnn.pth', map_location='mps'))
    model = deepcopy(model)
        
    print(f"Model size: {get_size(model):.2f} MB")
    
    # x, y = model.net_1, model.net_2
    # del model.net_1
    # del model.net_2
    
    # model.features = x
    # model.classifier = y
    
    return model

In [4]:
def write_to_file(pruning_amount, size):
    if 'results_again.csv' not in os.listdir():
        with open('results_again.csv', 'w') as f:
            f.write("Pruning Amount, Model Size\n")
    with open('results_again.csv', 'a') as f:
        f.write(f"{pruning_amount}, {size}\n")

In [5]:
def get_vgg_model():
    model = models.vgg11(weights=models.VGG11_Weights.IMAGENET1K_V1)
    x, y, z = model.avgpool, model.classifier, model.features

    model.avgpool = None
    model.classifier = None
    model.features = None

    del model.classifier
    del model.features
    del model.avgpool

    model.net_1 = z
    model.avgpool = x
    model.net_2 = y
    model.net_2[6] = torch.nn.Linear(4096, 33)
    
    return model

In [6]:
# pruner = PruningFineTuner(get_vgg_model())
# pruning_amount = 48

# model = pruner.prune(pruning_amount)
# print(f"Model size after pruning: {get_size(model):.2f} MB")

In [7]:
get_model()

Model size: 491.72 MB


SVCNN(
  (net_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  

In [8]:
# Create your original pruning amounts
pruning_amounts = np.arange(0, 100, 4)

# Center value
center = np.mean(pruning_amounts)

# Compute Gaussian-based probabilities
sigma = 15  # controls how "tight" around center you prefer; adjust if needed
probabilities = np.exp(-0.5 * ((pruning_amounts - center) / sigma) ** 2)
probabilities /= probabilities.sum()  # normalize to sum to 1

# Sample without replacement according to the probabilities
shuffled_pruning_amounts = np.random.choice(pruning_amounts, size=len(pruning_amounts), replace=False, p=probabilities)

# Now shuffled_pruning_amounts starts more around the middle
pruning_amounts = shuffled_pruning_amounts

In [9]:
pruning_amounts

array([36, 28, 44, 48, 76, 52, 16, 80, 20, 56, 40, 64, 72, 32, 68, 24, 60,
       88, 84,  4, 12, 96,  8, 92,  0])

In [10]:
print(f"Pruning amounts: {pruning_amounts}")
for pruning_amount in pruning_amounts:
    pruner = PruningFineTuner(get_model())
    model = pruner.prune(pruning_amount, True)
    print(f"Model size after {pruning_amount}% pruning: {get_size(model):.2f} MB")
    write_to_file(pruning_amount, get_size(model))
    pruner.reset()
    del model
    del pruner

Pruning amounts: [36 28 44 48 76 52 16 80 20 56 40 64 72 32 68 24 60 88 84  4 12 96  8 92
  0]
Model size: 491.72 MB
Total Filters to prune: 990 For Pruning Percentage: 36
Layers that will be pruned {0: 17, 6: 78, 16: 178, 13: 209, 18: 190, 11: 204, 8: 96, 3: 18}
Pruning Filters
Total Filters Pruned: 990
Model size after 36% pruning: 324.98 MB
PruningFineTuner object deleted and memory cleared.
Model size: 491.72 MB
Total Filters to prune: 770 For Pruning Percentage: 28
Layers that will be pruned {16: 153, 6: 79, 18: 158, 11: 149, 8: 69, 13: 138, 0: 11, 3: 13}
Pruning Filters
Total Filters Pruned: 770
Model size after 28% pruning: 353.60 MB
PruningFineTuner object deleted and memory cleared.
Model size: 491.72 MB
Total Filters to prune: 1210 For Pruning Percentage: 44
Layers that will be pruned {0: 24, 11: 263, 18: 221, 8: 112, 6: 110, 16: 213, 13: 239, 3: 28}
Pruning Filters


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x108533a10>>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


: 

: 